## Get resources

In [1]:
import coiled
import dask.distributed

In [2]:
cluster = coiled.Cluster(
    name="play", 
    n_workers=15, 
    configuration="coiled/default", 
    shutdown_on_close=False
)
client = dask.distributed.Client(cluster)

client

Using existing cluster: play


Client Scheduler: tls://ec2-18-191-218-174.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-18-191-218-174.us-east-2.compute.amazonaws.com:8787/status,Cluster Workers: 15 Cores: 60 Memory: 128.85 GB


In [8]:
cluster.scale(0)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [3]:
import dask.dataframe as dd
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
        "store_and_fwd_flag": "category",
        "PULocationID": "UInt16",
        "DOLocationID": "UInt16",    
        "payment_type": "UInt8",
    },
    blocksize="16 MiB",
)

data = df[["passenger_count", "trip_distance", "RatecodeID", "payment_type", "fare_amount"]]
data = data.fillna(0)

labels = (df.tip_amount / df.fare_amount) > 0.25
labels = labels.fillna(False)

from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, shuffle=True)
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [5]:
df.passenger_count.sum().compute()

131504474

## Train model

In [4]:
%%time

from sklearn.linear_model import SGDClassifier
from dask_ml.model_selection import HyperbandSearchCV
from scipy.stats import uniform, loguniform


clf = SGDClassifier(tol=1e-3, penalty='elasticnet', random_state=0)

params = {'alpha': loguniform(1e-2, 1e0),  # or np.logspace
          'l1_ratio': uniform(0, 1)}  # or np.linspace

search = HyperbandSearchCV(clf, params, max_iter=81, random_state=0)

search.fit(X_train, y_train, classes=[0, 1]);

TypeError: Cannot operate on Dask array with unknown chunk sizes. Use the following the compute chunk sizes:

   >>> X.compute_chunk_sizes()  # if Dask.Array
   >>> ddf.to_dask_array(lengths=True)  # if Dask.Dataframe

## Score

In [ ]:
search.score(X_test.sample(frac=0.1, random_state=123), y_test.sample(frac=0.1, random_state=123))

## What if we just sampled instead?

In [ ]:
clf.fit(
    X_train.sample(frac=0.01, random_state=123).compute(), 
    y_train.sample(frac=0.01, random_state=123).compute()
)

In [ ]:
clf.score(
    X_test.sample(frac=0.01, random_state=123).compute(), 
    y_test.sample(frac=0.01, random_state=123).compute()
)

Depending on our business needs, we maybe didn't need to do all of this :)